In [ ]:
mport pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
import keras.backend as K
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from itertools import product
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn

%matplotlib inline

In [ ]:
data = pd.read_csv(r'../../output/metrics.csv')
df = pd.DataFrame(data)
df['title'] = df['level_0']
attack_series = []
attack_IDs = []
dimensions_series = []
for _, row in df.iterrows():
    attack_series.append(row['level_1'].split('/')[-2])

df['attack'] = attack_series

for _, row in df.iterrows():
    dimension = int(row['attack'].split('_')[0].replace('p',''))
    dimensions_series.append(dimension)
    if row['attack'] in ['1080p', '720p', '480p', '360p', '240p', '144p']:
            attack_IDs.append(1)
    else:
        attack_IDs.append(0)

df['attack_ID'] = attack_IDs
df['dimension'] = dimensions_series
df = df.drop(['Unnamed: 0',
         'temporal_canny-series',
         'temporal_cross_correlation-series', 
         'temporal_difference-series', 
         'temporal_histogram_distance-series', 
         'temporal_histogram_distance-cosine',
         'level_0', 
          'dimension',
#           'temporal_canny-cosine',
#           'temporal_cross_correlation-cosine',
#               'temporal_difference-cosine',
         'level_1'],axis=1)

In [ ]:
df.head(25)

In [ ]:
df.describe()

In [ ]:
df_corr = df.corr()
plt.figure(figsize=(10,10))
corr = df_corr.corr('spearman')
corr.style.background_gradient().set_precision(2)

In [ ]:
train_prop = 0.8

df_1 = df[df['attack_ID'] == 1]
df_0 = df[df['attack_ID'] == 0]


num_train = int(df_1.shape[0]*0.8)
df_train = df_1[0:num_train]
df_test = df_1[num_train:]
df_attacks = df_0

df_train = df_train.sample(frac=1)
df_test = df_test.sample(frac=1)
df_attacks = df_attacks.sample(frac=1)

X_train = df_train.drop(['title',
                         'attack', 
                         'attack_ID',
                         'vmaf'],axis=1)
X_train = np.asarray(X_train)

X_test = df_test.drop(['title',  
                     'attack', 
                     'attack_ID',
                     'vmaf'],axis=1)
X_test = np.asarray(X_test)

X_attacks = df_attacks.drop(['title',  
                     'attack', 
                     'attack_ID',
                     'vmaf'],axis=1)

X_attacks = np.asarray(X_attacks)

print('Shape of train: {}'.format(X_train.shape))
print('Shape of test: {}'.format(X_test.shape))
print('Shape of attacks: {}'.format(X_attacks.shape))

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

x_train = ss.fit_transform(X_train)
x_test = ss.transform(X_test)
x_attacks = ss.transform(X_attacks)

variances = []
components = reversed(range(1,x_test.shape[1]+1))
for i in components:
    pca = PCA(n_components=i)
    pca.fit(x_train)
    variances.append(sum(pca.explained_variance_ratio_))
plt.plot(list(reversed(range(1,x_test.shape[1]+1))), variances)   
plt.grid()
plt.title('PCA')
plt.xlabel('Number of components')
plt.ylabel('Explained variance')

In [ ]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(x_train)
test_reduced = pca.transform(x_test)
attack_reduced = pca.transform(x_attacks)


In [ ]:
f, ax = plt.subplots(1,3, figsize=(25,10))
ax[0].set_title("Train set")
ax[1].set_title("Test set")
ax[2].set_title("Attack set")
ax[0].scatter(X_reduced[:,0], X_reduced[:,1], color='black')
ax[1].scatter(test_reduced[:,0], test_reduced[:,1], color='red')
ax[2].scatter(attack_reduced[:,0], attack_reduced[:,1], color='blue')

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(attack_reduced[:,0], attack_reduced[:,1], color='red', label='attack')
plt.scatter(X_reduced[:,0], X_reduced[:,1], color='green', label='Train')
plt.scatter(test_reduced[:,0], test_reduced[:,1], color='yellow', label='Test')
plt.legend()


In [ ]:
from sklearn import metrics 

nus = [0.1, 0.01, 0.001, 0.0001]

svm_results = pd.DataFrame(columns=['gamma', 'nu', 'n_components', 'train_acc', 'test_acc', 'attack_acc', 'model', 'auc'])
for n in reversed(range(1,x_test.shape[1]+1)):
    pca = PCA(n_components=n)
    X_reduced = pca.fit_transform(x_train)
    test_reduced = pca.transform(x_test)
    attack_reduced = pca.transform(x_attacks)

    gammas = [X_reduced.shape[1], 2*X_reduced.shape[1], X_reduced.shape[1]/2, 'auto']                                 
    for nu in nus:
        for gamma in gammas:
            classifier = svm.OneClassSVM(kernel='rbf',gamma=gamma, nu=nu, cache_size=5000)
            classifier.fit(X_reduced)
            y_pred_train = classifier.predict(X_reduced)
            y_pred_test = classifier.predict(test_reduced)
            y_pred_outliers = classifier.predict(attack_reduced)
            n_accurate_train = y_pred_train[y_pred_train == 1].size
            n_accurate_test = y_pred_test[y_pred_test == 1].size
            n_accurate_outliers = y_pred_outliers[y_pred_outliers == -1].size
            
            fpr, tpr, _ = metrics.roc_curve(np.concatenate([np.ones(y_pred_test.shape[0]),
                                                             -1*np.ones(y_pred_outliers.shape[0])]), 
                                              np.concatenate([y_pred_test, y_pred_outliers]) , pos_label=1)
            
            svm_results = svm_results.append({'nu': nu, 'gamma': gamma, 'n_components': n, 'train_acc': n_accurate_train/X_reduced.shape[0],
                           'test_acc': n_accurate_train/X_reduced.shape[0], 'attack_acc': n_accurate_outliers/attack_reduced.shape[0],
                           'model': 'svm', 'auc': metrics.auc(fpr, tpr)}, ignore_index=True)


In [ ]:
svm_results.sort_values('auc', ascending=False).head(10)

In [ ]:
# Let's create a metric in order to find a best model from that metric
# We will get just accuracies from test y attack above 0.7, and will add them

In [ ]:
def metric(row, th=0.7):
    if row['test_acc'] < th or row['attack_acc'] < th:
        return 0
    else:
        return row['test_acc'] + row['attack_acc']

In [ ]:
svm_results['score'] = svm_results.apply(metric, axis=1)

In [ ]:
svm_results[svm_results['test_acc'] > svm_results['attack_acc']].sort_values('score', ascending=False).head(10)

In [ ]:
from sklearn.ensemble import IsolationForest

estimators = [50, 100, 150, 200, 500]
contaminations = [0.01, 0.05, 0.1]
isolation_results = pd.DataFrame(columns=['estimators', 'contamination', 'n_components', 'max_features',
                                          'test_acc', 'attack_acc', 'model', 'auc'])
for n in reversed(range(1,x_test.shape[1]+1)):
    pca = PCA(n_components=n)
    X_reduced = pca.fit_transform(x_train)
    test_reduced = pca.transform(x_test)
    attack_reduced = pca.transform(x_attacks)
    max_features = list(range(1, n + 1))
    for estimator in estimators:
        for contamination in contaminations:
            for max_feature in max_features:
                classifier = IsolationForest(n_estimators=estimator,
                                             contamination=contamination,
                                             max_features=max_feature,
                                            n_jobs=5)
                
                classifier.fit(X_reduced)
                y_pred_train = classifier.predict(X_reduced)
                y_pred_test = classifier.predict(test_reduced)
                y_pred_outliers = classifier.predict(attack_reduced)
                n_error_train = y_pred_train[y_pred_train == 1].size
                n_error_test = y_pred_test[y_pred_test == 1].size
                n_error_outliers = y_pred_outliers[y_pred_outliers == -1].size
                
                fpr, tpr, _ = metrics.roc_curve(np.concatenate([np.ones(y_pred_test.shape[0]),
                                                 -1*np.ones(y_pred_outliers.shape[0])]), 
                                  np.concatenate([y_pred_test, y_pred_outliers]) , pos_label=1)

                isolation_results = isolation_results.append({'estimators': estimator, 'contamination': contamination,
                                                              'n_components': n, 'max_features': max_feature,
                                                              'train_acc': n_error_train/X_reduced.shape[0],
                                                              'test_acc': n_error_train/X_reduced.shape[0],
                                                              'attack_acc': n_error_outliers/attack_reduced.shape[0],
                                                              'model': 'isolation_forest',
                                                              'auc': metrics.auc(fpr, tpr)}, ignore_index=True)

In [ ]:
isolation_results['score'] = isolation_results.apply(metric, axis=1)

In [ ]:
isolation_results[isolation_results['test_acc'] > isolation_results['attack_acc']].sort_values('test_acc', ascending=False).head(10)

In [ ]:
isolation_results.sort_values('auc', ascending=False).head(10)

# Let's plot ROCs

In [ ]:
def plot_roc(fpr, tpr, title):
    roc_auc = metrics.auc(fpr, tpr)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
            lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic: {}'.format(title))
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
pca = PCA(n_components=4)
X_reduced = pca.fit_transform(x_train)
test_reduced = pca.transform(x_test)
attack_reduced = pca.transform(x_attacks)


classifier = svm.OneClassSVM(kernel='rbf',gamma=2, nu=0.001, cache_size=5000)
classifier.fit(X_reduced)
y_pred_train = classifier.predict(X_reduced)
y_pred_test = classifier.predict(test_reduced)
y_pred_outliers = classifier.predict(attack_reduced)
n_accurate_train = y_pred_train[y_pred_train == 1].size
n_accurate_test = y_pred_test[y_pred_test == 1].size
n_accurate_outliers = y_pred_outliers[y_pred_outliers == -1].size

fpr, tpr, _ = metrics.roc_curve(np.concatenate([np.ones(y_pred_test.shape[0]),
                                                 -1*np.ones(y_pred_outliers.shape[0])]), 
                                  np.concatenate([y_pred_test, y_pred_outliers]) , pos_label=1)

plot_roc(fpr, tpr, 'One Class SVM')

In [ ]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(x_train)
test_reduced = pca.transform(x_test)
attack_reduced = pca.transform(x_attacks)


classifier = IsolationForest(n_estimators=500,
                            contamination=0.1,
                            max_features=2,
                            n_jobs=5)
classifier.fit(X_reduced)
y_pred_train = classifier.predict(X_reduced)
y_pred_test = classifier.predict(test_reduced)
y_pred_outliers = classifier.predict(attack_reduced)
n_accurate_train = y_pred_train[y_pred_train == 1].size
n_accurate_test = y_pred_test[y_pred_test == 1].size
n_accurate_outliers = y_pred_outliers[y_pred_outliers == -1].size

fpr, tpr, _ = metrics.roc_curve(np.concatenate([np.ones(y_pred_test.shape[0]),
                                                 -1*np.ones(y_pred_outliers.shape[0])]), 
                                  np.concatenate([y_pred_test, y_pred_outliers]) , pos_label=1)

plot_roc(fpr, tpr, 'Isolation Forest')